In [1]:
%pip install pandas langchain-community langchain unidecode nltk

Note: you may need to restart the kernel to use updated packages.


In [18]:
import re
from unidecode import unidecode
from nltk.corpus import stopwords

# baixa as stopwords em pt-br
stopwords_list = set(stopwords.words('portuguese'))

def normalize_text(text):
    # deixa tudo minúsculo
    text = text.lower()
    
    text = unidecode(text) # remove os acentos codificados

    # remove espaços desnecessários
    text = re.sub(r'\s+', ' ', text).strip()
    
    # remove pontuação
    text = re.sub(r'[^\w\s]', '', text)
    
    return text

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords_list]) # remove as palavras do texto que estiverem na lista de stopwords

In [20]:
import os
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# define as colunas do csv que serão usadas
useful_data = ['product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'overall_rating', 'review_text']

# carrega o csv
def load_data(file_path):
    # lê o arquivo csv
    df = pd.read_csv(file_path)
    
    # exclui as colunas que não serão usadas
    df_reduced = df.drop(columns=[col for col in df.columns if col not in useful_data])

    # limpa e normaliza o texto
    for column in useful_data:
        df_reduced[column] = df_reduced[column].apply(lambda x: normalize_text(str(x)))
        df_reduced[column] = df_reduced[column].apply(lambda x: remove_stopwords(str(x)))

    # define o nome e o caminho do arquivo de saída
    result_file_name = 'data_processed.csv'  # nome do arquivo de saída
    new_file_path = os.path.join('data', result_file_name)  # caminho para salvar o arquivo de saída na pasta 'data'
    
    # salva os dados em csv
    df_reduced.to_csv(new_file_path, index=False)

    return new_file_path

def load_and_chunk(file_path):
    # gera os documentos com o csv processado
    loader = CSVLoader(file_path=file_path, encoding='utf-8', csv_args={
        'delimiter': ',', 'quotechar': '"', 'fieldnames': useful_data
    }) # classe do langchain para manipular os dados
    # delimiter define o que separa as colunas do csv (no caso, uma vírgula)
    # quotechar define o caractere que envolve strings, permitindo que uma string tenha vírgulas sem ser divididas em colunas
    # fieldnames são os nomes das colunas que serão carregadas

    docs = loader.load() # retorna os documentos gerados

    # chunkeniza e adiciona overlap nos dados
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
    splits = text_splitter.split_documents(docs)

    # Retorna os chunks gerados
    print(f"Total de chunks gerados: {len(splits)}")  # Imprime a quantidade total de chunks
    for i, chunk in enumerate(splits):
        print(f"Chunk {i+1}: {chunk}")  # Imprime cada chunk gerado

    return splits

if __name__ == "__main__":
    # Caminho para o arquivo CSV a ser carregado
    file_path = r'H:\Github\Fatec\atmchat\data\chat_data.csv'  # Substitua pelo caminho do seu arquivo
    
    # Carrega os dados e chunks
    print("Iniciando o processamento do arquivo CSV...")
    processed_file_path = load_data(file_path)
    print(f"Dados processados salvos em: {processed_file_path}")
    
    # Carrega e chunkeniza os dados
    splits = load_and_chunk(processed_file_path)
    print("Processamento concluído.")


Iniciando o processamento do arquivo CSV...
Dados processados salvos em: data\data_processed.csv
Total de chunks gerados: 125
Chunk 1: page_content='product_name: product_name
product_brand: product_brand
site_category_lv1: site_category_lv1
site_category_lv2: site_category_lv2
overall_rating: overall_rating
review_text: review_text' metadata={'source': 'data\\data_processed.csv', 'row': 0}
Chunk 2: page_content='product_name: notebook asus vivobook max x541nago472t intel celeron quad core 4gb 500gb tela led 156 windows 10 branco
product_brand: nan
site_category_lv1: informatica
site_category_lv2: notebook
overall_rating: 4
review_text: contente compra entrega rapida unico problema americanas troca devolucao produto consumidor problemas espera' metadata={'source': 'data\\data_processed.csv', 'row': 1}
Chunk 3: page_content='product_name: copo acrilico canudo 500ml rocie
product_brand: nan
site_category_lv1: utilidades domesticas
site_category_lv2: copos tacas canecas
overall_rating: 4
